In [ ]:
import os 
import json
from openai import OpenAI
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
import ollama

In [ ]:
MODEL_GPT = 'gpt-4.1-mini'
MODEL_OLLAMA = 'llama3.2'

In [ ]:
#setup environment variables   
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("API key is not valid")


In [ ]:
# Technical Question Explainer — choose input mode below
USE_INPUT = False  # Set to True to type your question when you run this cell; False = use hardcoded question

DEFAULT_QUESTION = """
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""

if USE_INPUT:
    question = input("Enter your technical question (or press Enter for default): ").strip()
    if not question:
        question = DEFAULT_QUESTION.strip()
else:
    question = DEFAULT_QUESTION

print("Question to analyze:")
print(question)

# Prompts and messages for the model
system_prompt = "You are a helpful technical tutor who answers questions about Python code, software engineering, data science, and LLMs."
user_prompt = "Please give a detailed explanation to the following question: " + question

messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt}
]

In [ ]:
# Get answer from OpenAI with streaming
client = OpenAI()  # uses OPENAI_API_KEY from environment

stream = client.chat.completions.create(
    model=MODEL_GPT,
    messages=messages,
    stream=True
)

response = ""
display_handle = display(Markdown(""), display_id=True)

for chunk in stream:
    if chunk.choices[0].delta.content:
        response += chunk.choices[0].delta.content
        update_display(Markdown(f"## OpenAI ({MODEL_GPT}) Response:\n\n{response}"), display_id=display_handle.display_id)

In [ ]:
# Get answer from Ollama (local) with streaming
ollama_response = ""
ollama_handle = display(Markdown(""), display_id=True)

for chunk in ollama.chat(model=MODEL_OLLAMA, messages=messages, stream=True):
    part = chunk.get("message", {}).get("content", "") or ""
    if part:
        ollama_response += part
        update_display(Markdown(f"## Ollama ({MODEL_OLLAMA}) Response:\n\n{ollama_response}"), display_id=ollama_handle.display_id)